In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers datasets accelerate
from transformers import T5Tokenizer, T5ForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
from datasets import load_dataset, Dataset
import json
import torch

  Attempting uninstall: nvidia-cuda-runtime-cu12
    Found existing installation: nvidia-cuda-runtime-cu12 12.5.82
    Uninstalling nvidia-cuda-runtime-cu12-12.5.82:
      Successfully uninstalled nvidia-cuda-runtime-cu12-12.5.82
  Attempting uninstall: nvidia-cuda-nvrtc-cu12
    Found existing installation: nvidia-cuda-nvrtc-cu12 12.5.82
    Uninstalling nvidia-cuda-nvrtc-cu12-12.5.82:
      Successfully uninstalled nvidia-cuda-nvrtc-cu12-12.5.82
  Attempting uninstall: nvidia-cuda-cupti-cu12
    Found existing installation: nvidia-cuda-cupti-cu12 12.5.82
    Uninstalling nvidia-cuda-cupti-cu12-12.5.82:
      Successfully uninstalled nvidia-cuda-cupti-cu12-12.5.82
  Attempting uninstall: nvidia-cublas-cu12
    Found existing installation: nvidia-cublas-cu12 12.5.3.2
    Uninstalling nvidia-cublas-cu12-12.5.3.2:
      Successfully uninstalled nvidia-cublas-cu12-12.5.3.2
  Attempting uninstall: nvidia-cusparse-cu12
    Found existing installation: nvidia-cusparse-cu12 12.5.1.3
    Unins

In [ ]:
with open('/content/flans3_final_v2.json') as f:
    raw_data = json.load(f)

In [ ]:
!pip install transformers datasets torch

In [ ]:
import json
from datasets import Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import DataCollatorForSeq2Seq

In [ ]:
model_name = "hyrinmansoor/text2frappe-s3-flan-query"
tokenizer = T5Tokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/723 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq
)
from datasets import Dataset
import json

# # ✅ Load your dataset
# with open("/content/drive/MyDrive/Changai/S3/Datasets/S3_flan_old.json") as f:
#     raw_data = json.load(f)

INSTRUCTION = "Generate the correct Frappe query for the given question, using the provided doctype and fields."

# ✅ Prepare prompt-based data
processed_data = []
for entry in raw_data:
    try:
        input_data = entry["input"]
        if not isinstance(input_data, dict):
            continue

        prompt = (
            f"Instruction: {INSTRUCTION}\n"
            f"Doctype: {input_data['doctype']}\n"
            f"Question: {input_data['question']}\n"
            f"Fields: {input_data['fields']}"
        )

        processed_data.append({
            "input": prompt,
            "output": entry["output"]
        })
    except Exception as e:
        print("Skipping due to error:", e)
        continue

# ✅ Convert to Hugging Face Dataset
dataset = Dataset.from_list(processed_data)
dataset = dataset.train_test_split(test_size=0.1)


model_name = "hyrinmansoor/text2frappe-s3-flan-query"
tokenizer = T5Tokenizer.from_pretrained(model_name)


# ✅ Load and resize model to accommodate new tokens
model = T5ForConditionalGeneration.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))

# ✅ Tokenize with `{}` token loss reinforcement
def tokenize_and_enhance_loss(example):
    model_inputs = tokenizer(example["input"], max_length=512, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(example["output"], max_length=128, truncation=True, padding="max_length")

    # Reinforce loss on `{` and `}`
    target_ids = labels["input_ids"]
    for i, token_id in enumerate(target_ids):
        if token_id == tokenizer.pad_token_id:
            target_ids[i] = -100  # Mask pad tokens
        if token_id in [tokenizer.convert_tokens_to_ids('{'), tokenizer.convert_tokens_to_ids('}')]:
            continue  # Keep loss for curly braces

    model_inputs["labels"] = target_ids
    return model_inputs

# ✅ Tokenize dataset
tokenized_datasets = dataset.map(tokenize_and_enhance_loss, batched=False)

# ✅ Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/Changai/S3/Model/text2frappe-s3-flan_18_08_25_3",
    eval_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=16,
    predict_with_generate=True,
    logging_dir="./logs",
    logging_steps=100
)

# ✅ Data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# ✅ Trainer setup
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# ✅ Train and Save
trainer.train()
trainer.evaluate()
model.save_pretrained("/content/drive/MyDrive/Changai/S3/Model/text2frappe-s3-flan_18_08_25_3")
tokenizer.save_pretrained("/content/drive/MyDrive/Changai/S3/Model/text2frappe-s3-flan_18_08_25_3")

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

Map:   0%|          | 0/4644 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:4006: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: hyrin (hyrin-erpgulf) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,0.113400,0.089310
2,0.090800,0.073354
3,0.067500,0.064264
4,0.057800,0.060665
5,0.050300,0.058491
6,0.052300,0.058391
7,0.039400,0.057797
8,0.034600,0.058024
9,0.036200,0.055741
10,0.034900,0.054950


('/content/drive/MyDrive/Changai/S3/Model/text2frappe-s3-flan_18_08_25_3/tokenizer_config.json',
 '/content/drive/MyDrive/Changai/S3/Model/text2frappe-s3-flan_18_08_25_3/special_tokens_map.json',
 '/content/drive/MyDrive/Changai/S3/Model/text2frappe-s3-flan_18_08_25_3/spiece.model',
 '/content/drive/MyDrive/Changai/S3/Model/text2frappe-s3-flan_18_08_25_3/added_tokens.json')

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load tokenizer and model
model_path = "/content/drive/MyDrive/Changai/S3/Model/text2frappe-s3-flan_18_08_25_3"
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)
prompts = [
    # Sales Invoice
    {
        "doctype": "Sales Invoice",
        "question": "List all sales invoices for customer 'ABC Ltd'.",
        "fields": ["customer"]
    },
    {
        "doctype": "Sales Invoice",
        "question": "Find invoices with total greater than 1000.",
        "fields": ["total"]
    },
    # Purchase Invoice
    {
        "doctype": "Purchase Invoice",
        "question": "List invoices for supplier 'XYZ Pvt Ltd'.",
        "fields": ["supplier"]
    },
    {
        "doctype": "Purchase Invoice",
        "question": "Show invoices posted in July 2025.",
        "fields": ["posting_date"]
    },
    # Customer
    {
        "doctype": "Customer",
        "question": "List all customers in 'Retail' group.",
        "fields": ["customer_group"]
    },
    {
        "doctype": "Customer",
        "question": "Find customers in territory 'North'.",
        "fields": ["territory"]
    },
    # Employee
    {
        "doctype": "Employee",
        "question": "List employees in department 'Sales'.",
        "fields": ["department"]
    },
    {
        "doctype": "Employee",
        "question": "Find employees with designation 'Manager'.",
        "fields": ["designation"]
    },
    # Item
    {
        "doctype": "Item",
        "question": "List items in item group 'Electronics'.",
        "fields": ["item_group"]
    },
    {
        "doctype": "Item",
        "question": "Find items with valuation rate above 5000.",
        "fields": ["valuation_rate"]
    },
    {
        "doctype": "Sales Invoice",
        "question": "Get all sales invoices for customer 'ABC Ltd'.",
        "fields": ["customer"]
    },
    {
        "doctype": "Sales Invoice",
        "question": "Find invoices with total more than 1000.",
        "fields": ["total"]
    },

    # Purchase Invoice
    {
        "doctype": "Purchase Invoice",
        "question": "Get all purchase invoices for supplier 'XYZ Pvt Ltd'.",
        "fields": ["supplier"]
    },
    {
        "doctype": "Purchase Invoice",
        "question": "Show invoices posted in July 2025.",
        "fields": ["posting_date"]
    },

    # Customer
    {
        "doctype": "Customer",
        "question": "List customers in group 'Retail'.",
        "fields": ["customer_group"]
    },
    {
        "doctype": "Customer",
        "question": "Find customers in 'North' territory.",
        "fields": ["territory"]
    },

    # Employee
    {
        "doctype": "Employee",
        "question": "List employees in 'Sales' department.",
        "fields": ["department"]
    },
    {
        "doctype": "Employee",
        "question": "Find employees with designation 'Manager'.",
        "fields": ["designation"]
    },

    # Item
    {
        "doctype": "Item",
        "question": "List items in group 'Electronics'.",
        "fields": ["item_group"]
    },
    {
        "doctype": "Item",
        "question": "Find items with price above 5000.",
        "fields": ["valuation_rate"]
    },

    # Supplier
    {
        "doctype": "Supplier",
        "question": "List suppliers in country 'India'.",
        "fields": ["country"]
    },
    {
        "doctype": "Supplier",
        "question": "Find suppliers with status 'Active'.",
        "fields": ["status"]
    },

    # Project
    {
        "doctype": "Project",
        "question": "List projects in 'Construction' category.",
        "fields": ["project_name", "project_type"]
    },
    {
        "doctype": "Project",
        "question": "Get projects owned by 'John Doe'.",
        "fields": ["owner"]
    },
    # Sales Invoice
    {
        "doctype": "Sales Invoice",
        "question": "Get all sales invoices.",
        "fields": ["name"]
    },
    {
        "doctype": "Sales Invoice",
        "question": "Count total sales invoices.",
        "fields": ["name"]
    },

    # Purchase Invoice
    {
        "doctype": "Purchase Invoice",
        "question": "Get all purchase invoices.",
        "fields": ["name"]
    },
    {
        "doctype": "Purchase Invoice",
        "question": "Count total purchase invoices.",
        "fields": ["name"]
    },

    # Customer
    {
        "doctype": "Customer",
        "question": "Get all customers.",
        "fields": ["name"]
    },
    {
        "doctype": "Customer",
        "question": "Count total customers.",
        "fields": ["name"]
    },

    # Employee
    {
        "doctype": "Employee",
        "question": "Get all employees.",
        "fields": ["name"]
    },
    {
        "doctype": "Employee",
        "question": "Count total employees.",
        "fields": ["name"]
    },

    # Item
    {
        "doctype": "Item",
        "question": "Get all items.",
        "fields": ["name"]
    },
    {
        "doctype": "Item",
        "question": "Count total items.",
        "fields": ["name"]
    },

    # Supplier
    {
        "doctype": "Supplier",
        "question": "Get all suppliers.",
        "fields": ["name"]
    },
    {
        "doctype": "Supplier",
        "question": "Count total suppliers.",
        "fields": ["name"]
    }
]




for idx, item in enumerate(prompts, 1):
    test_prompt = f"""
Instruction: Generate the correct Frappe query for the given question, using the provided doctype and fields.
Doctype: {item['doctype']}
Question: {item['question']}
Fields: {item['fields']}
"""
    inputs = tokenizer(test_prompt.strip(), return_tensors="pt")
    output_ids = model.generate(
        input_ids=inputs["input_ids"],
        max_length=128,
        do_sample=False,
        num_beams=4,
        early_stopping=True
    )
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    # output_text=output_text.replace("[BT]","`")

    # output_text = output_text.replace("<pad>", "").replace("</s>", "").strip()

    print(f"\n🔢 Example {idx}")
    print("🗒️ Prompt:\n", test_prompt.strip())
    print("📄 Generated Query:\n", output_text)



🔢 Example 1
🗒️ Prompt:
 Instruction: Generate the correct Frappe query for the given question, using the provided doctype and fields.
Doctype: Sales Invoice
Question: List all sales invoices for customer 'ABC Ltd'.
Fields: ['customer']
📄 Generated Query:
 frappe.db.sql('''SELECT customer FROM tabSales Invoice WHERE customer = 'ABC Ltd' ''', as_dict=True)

🔢 Example 2
🗒️ Prompt:
 Instruction: Generate the correct Frappe query for the given question, using the provided doctype and fields.
Doctype: Sales Invoice
Question: Find invoices with total greater than 1000.
Fields: ['total']
📄 Generated Query:
 frappe.db.sql('''SELECT total FROM tabSales Invoice WHERE total > 1000''', as_dict=True)

🔢 Example 3
🗒️ Prompt:
 Instruction: Generate the correct Frappe query for the given question, using the provided doctype and fields.
Doctype: Purchase Invoice
Question: List invoices for supplier 'XYZ Pvt Ltd'.
Fields: ['supplier']
📄 Generated Query:
 frappe.db.sql('SELECT supplier FROM [BT]tabPurcha

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_path = "/content/drive/MyDrive/Changai/S3_test"
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)

# 🔍 Sample test input
test_prompt = """Instruction: Generate the correct Frappe query for the given question, using the provided doctype and fields.
Doctype: Company
Question: Show me company details where the name is 'France'.
Fields: name, country"""

inputs = tokenizer(test_prompt, return_tensors="pt")

output_ids = model.generate(
    input_ids=inputs["input_ids"],
    max_length=128,
    do_sample=False,
    num_beams=4,
    early_stopping=True
)

# ✅ Decode and print output
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=False)
output_text=output_text.replace("<pad>","")
output_text=output_text.replace("</s>","")
print("📄 Generated Query:\n", output_text)


In [ ]:
model_path="/content/drive/MyDrive/Changai/S3_test"
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained(model_path)

In [ ]:
from huggingface_hub import HfApi, HfFolder, notebook_login
notebook_login()

In [ ]:
from huggingface_hub import create_repo

create_repo("text2frappe-s3-flan-query", private=True)

In [ ]:
from huggingface_hub import upload_folder

upload_folder(
    repo_id="hyrinmansoor/text2frappe-s3-flan-query",
    folder_path=model_path,
    path_in_repo=".",
    repo_type="model"
)


In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

model = T5ForConditionalGeneration.from_pretrained('hyrinmansoor/text2frappe-s3-flan-query')
tokenizer = T5Tokenizer.from_pretrained('hyrinmansoor/text2frappe-s3-flan-query')
test_cases = [
    """Instruction: Generate the correct Frappe query for the given question, using the provided doctype and fields.
Doctype: Company
Fields: company_name
Question: What's the full name of our primary registered company named 'HTS Pvt Ltd'?""",

    """Instruction: Generate the correct Frappe query for the given question, using the provided doctype and fields.
Doctype: Company
Fields: parent_company
Question: Who is listed as the parent company of 'FusionCorp International'?""",

    """Instruction: Generate the correct Frappe query for the given question, using the provided doctype and fields.
Doctype: Finance Book
Fields: finance_book_name
Question: Can I rename or update the name of an existing finance book?"""
]

# Define generation function
def generate_query(test_input):
    inputs = tokenizer(test_input, return_tensors="pt", max_length=512, truncation=True, padding=True)
    outputs = model.generate(inputs['input_ids'], max_length=512, num_beams=5, early_stopping=True)
    decoded= tokenizer.decode(outputs[0], skip_special_tokens=False)
    decoded=output_text.replace("<pad>","")
    decoded=output_text.replace("</s>","")
    return decoded


# Run test cases
for i, test_input in enumerate(test_cases, start=1):
    print(f"\n--- Test Case {i} ---")
    print(f"Input:\n{test_input}")
    print("\nGenerated Query:")
    print(generate_query(test_input))
    print("\n")
